In [82]:
import pandas as pd
from gurobipy import *
import numpy as np


In [94]:
# Reading the data from the excel sheets
source_df = pd.read_excel("Source_facility_info.xlsx", sheet_name="PZA")
destination_df = pd.read_excel("Destination_facility_info.xlsx", sheet_name="PZE")
trucking_df = pd.read_excel("Trucking_info.xlsx", sheet_name="Truck")

# Convert relevant columns to datetime and normalize shift times
def normalize_shift_times(row):
    start_hour = row['Start of shift'].hour + row['Start of shift'].minute / 60
    end_hour = row['End of lay-on'].hour + row['End of lay-on'].minute / 60
    if end_hour < start_hour:
        end_hour += 24  # normalize end_hour for the next day
    return pd.Series([start_hour, end_hour])

destination_df[['Start of shift', 'End of lay-on']] = destination_df.apply(normalize_shift_times, axis=1)
source_df['planned_end_of_loading'] = pd.to_datetime(source_df['planned_end_of_loading'])



In [95]:
# Initialize the model
model = Model("DHL_Optimization")

# Decision Variables
source_list = list(trucking_df['Origin_ID'].unique()[:5])  # Index of PZA
destination_list = list(trucking_df['Destination_ID'].unique()[:5])  # Index of PZE
routes_list = [(i, j) for i in source_list for j in destination_list if i != j]
consignment_list = [
    x for (i, j) in routes_list
    for x in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values
]

valid_combinations = [(i, j, k) for (i, j) in routes_list for k in consignment_list if k in source_df[(source_df['Origin_ID'] == i) & (source_df['Destination_ID'] == j)]['id'].values]

trucks = range(300)

# Decision Variables
X = {}
Z = {}
T = {}
ArrivalDay = {}
ArrivalTime = {}
ArrivalDayBinary = {}

for (i, j, k, l) in [(i, j, k, l) for (i, j, k) in valid_combinations for l in trucks]:
    X[(i, j, k, l)] = model.addVar(vtype=GRB.BINARY, name=f"X_{i}_{j}_{k}_{l}")

for l in trucks:
    Z[l] = model.addVar(vtype=GRB.BINARY, name=f"Z_{l}")
    T[l] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"T_{l}")
    ArrivalDay[l] = model.addVar(lb = 0, ub=3, vtype=GRB.INTEGER, name=f"ArrivalDay_{l}")
    ArrivalTime[l] = model.addVar(lb=0, ub=24, vtype=GRB.CONTINUOUS, name=f"ArrivalTime_{i}_{j}_{k}_{l}")

    for d in range(4):  # Maximum number of days to consider
        ArrivalDayBinary[(l, d)] = model.addVar(vtype=GRB.BINARY, name=f"ArrivalDayBinary_{l}_{d}")


In [96]:
# Objective function: Minimize the total arrival time
model.setObjective(
    quicksum(X[i, j, k, l] for (i, j, k, l) in [(i, j, k, l) for (i, j, k) in valid_combinations for l in trucks]) 
    - quicksum(Z[l] for l in trucks) 
    - quicksum(ArrivalDay[l] for l in trucks), 
    GRB.MAXIMIZE
)

In [97]:
# Constraints
# 1. Each truck can carry at most 2 consignments
for l in trucks:
    model.addConstr(quicksum(X[(i, j, k, l)] for (i, j, k) in valid_combinations) <= 2 * Z[l])

In [98]:
# 2. Consignment can only be released after the latest release time of the consignments
for (i, j, k) in valid_combinations:
    release_time = source_df[source_df['id'] == k]['planned_end_of_loading'].dt.hour.values[0]  # Convert to hours
    for l in trucks:
        model.addConstr(T[l] >= release_time * X[(i, j, k, l)])

In [99]:
# Constraint 3: Truck must arrive at the destination within the operational hours
for (i, j, k) in valid_combinations:
    start_shift = destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    end_shift = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0]
    travel_time = trucking_df[(trucking_df['Origin_ID'] == i) & (trucking_df['Destination_ID'] == j)]['OSRM_time [sek]'].values[0] / 3600  # Convert to hours
    # print(f"Route {i}->{j}->{k} | Start Shift: {start_shift}, End Shift: {end_shift}, Travel Time: {travel_time}")
    for l in trucks:
        ArrivalTime[(l)] = (T[l] + travel_time * X[(i, j, k, l)] + 24 * ArrivalDay[l]) - 24 * model.addVar(vtype=GRB.INTEGER, name=f"multiplier_{i}_{j}_{k}_{l}")
        model.addConstr(ArrivalTime[(l)] >= start_shift)
        model.addConstr(ArrivalTime[(l)] <= end_shift)

In [100]:
# 4. Each consignment must be assigned to exactly one truck
for (i, j, k) in valid_combinations:
    model.addConstr(quicksum(X[(i, j, k, l)] for l in trucks if (i, j, k) in valid_combinations) == 1)

In [90]:
# M=4
# for l in trucks:
#     for d in range(4):  # Maximum number of days to consider
#         # model.addConstr((ArrivalDay[l] == d) >> (ArrivalDayBinary[(l, d)] == 1))
#         # model.addConstr((ArrivalDay[l] != d) >> (ArrivalDayBinary[(l, d)] == 0))
#         model.addConstr(ArrivalDay[l] - d <= M * (1 - ArrivalDayBinary[(l, d)]))
#         model.addConstr(ArrivalDay[l] - d >= 1 - M * ArrivalDayBinary[(l, d)])

In [101]:
for j in destination_list:
    # working_hours = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0] - destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
    sorting_capacity_per_day =  destination_df[destination_df['Destination_ID'] == j]['Sorting capacity'].values[0]
    print(sorting_capacity_per_day)
    for d in range(4):  # Maximum number of days to consider
        model.addConstr(
            quicksum(X[(i, j, k, l)] * source_df[source_df['id'] == k]['Consignment quantity'].values[0] * ArrivalDayBinary[(l, d)]
                     for i in source_list if j != i
                     for k in consignment_list
                     for l in trucks
                     if (i, j, k) in valid_combinations) <= sorting_capacity_per_day,
            name=f"SortingCapacity_{j}_{d}"
        )


15000
15000
30000
30000
40000


In [92]:
# model.setParam('TimeLimit', 100)

In [102]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i5-1240P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 430978 rows, 289200 columns and 1721100 nonzeros
Model fingerprint: 0x8c56451d
Model has 20 quadratic constraints
Variable types: 600 continuous, 288600 integer (144900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [2e+02, 2e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 2e+01]
  RHS range        [1e+00, 3e+01]
  QRHS range       [2e+04, 4e+04]
Presolve removed 0 rows and 300 columns
Presolve time: 3.60s
Presolved: 1004598 rows, 862500 columns, 4015500 nonzeros
Variable types: 300 continuous, 862200 integer (718500 binary)
Found heuristic solution: objective 239.0000000
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log

In [107]:
# Accessing ArrivalDay values after optimization
if model.status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.SUBOPTIMAL]:
    # Extract the data into a DataFrame
    data = []
    k_set = set()
    for (i, j, k, l) in X.keys():
        if (Z[l].X == 1 ) and (X[i,j,k,l].X == 1.0):
            start_shift = destination_df[destination_df['Destination_ID'] == j]['Start of shift'].values[0]
            end_shift = destination_df[destination_df['Destination_ID'] == j]['End of lay-on'].values[0]
            travel_time = trucking_df[(trucking_df['Origin_ID'] == i) & (trucking_df['Destination_ID'] == j)]['OSRM_time [sek]'].values[0] / 3600
            arrival_time = T[l].X + travel_time
            data.append({
                'Origin(PZA)': i,
                'Destination (PZE)': j,
                'Consignment ID': k,
                'Truck Id': l,
                'Departure time': T[l].X,
                'Arrival Day': ArrivalDay[l].X,
                'Arrival Time': ArrivalTime[l].X11111111
    1-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------qqqwerastd(zf)x_o
    ":",
                "Destination Start Shift": start_shift,
                "Destination End Shift": end_shift,
                "Travel Time": travel_time
                
          })

AttributeError: 'gurobipy.LinExpr' object has no attribute 'X'

In [104]:
df = pd.DataFrame(data)

In [106]:
import os
path = os.getcwd()
df.to_csv("output/output.csv")